In [ ]:
#-- Apply Some Augmentation on a video and Save results as a video --

In [ ]:
#-- Imports ------------------------------------------------------------------------------------------------------
import cv2
import matplotlib.pyplot as plt

import numpy as np
import random

import os
import shutil
import glob
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Initialize ---------------------------------------------------------------------------------------------------
ds_fight_input_path = '/kaggle/input/sample-videos-for-fight-detection-2/fight/'
ds_normal_input_path = '/kaggle/input/sample-videos-for-fight-detection-2/normal/'
result_dir = '/kaggle/working/results/'
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Add Noise ----------------------------------------------------------------------------------------
def add_noise(image, factor):
    noise = np.random.normal(0, factor, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Change Brightness --------------------------------------------------------------------------------
def adjust_brightness(image, factor):
    return cv2.convertScaleAbs(image, alpha=factor, beta=0)
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Add Noise and Change Brightness ------------------------------------------------------------------
def apply_noise_and_brightness(image, nois_factor, bright_factor):
    noisy_image = add_noise(image, nois_factor)     
    return adjust_brightness(noisy_image, bright_factor)
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Funvtion to Apply Blur ---------------------------------------------------------------------------------------
def apply_blur(image, kernel_size=(7, 7)):    
    return cv2.GaussianBlur(image, kernel_size, 0)
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#--function to transform video -----------------------------------------------------------------------------------
def transform_video(input_video_path, output_video_path, nois_factor=None, bright_factor=None, blur=False):
    
    cap = cv2.VideoCapture(input_video_path)    
   
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))    
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    #-- Process the video frame by frame --
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break 
        
        #-- Apply filters if selected --
        if nois_factor is not None and bright_factor is not None:
            frame = apply_noise_and_brightness(frame, nois_factor, bright_factor)
        elif nois_factor is not None:
            frame = add_noise(frame, nois_factor)
        elif bright_factor is not None:
            frame = adjust_brightness(frame, bright_factor)
        
        if blur:
            frame = apply_blur(frame)

        #-- Write the processed frame to the output video --
        out.write(frame)    
    
    cap.release()
    out.release()    
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Create Result Folder ----------------------------------------------------------------------------------------
fight_results_dir = result_dir + 'fight/'
normal_results_dir = result_dir + 'normal/'

os.makedirs(fight_results_dir, exist_ok=True)
os.makedirs(normal_results_dir, exist_ok=True)
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Apply Augmentations on Fight Samples -------------------------------------------------------------------------
fight_video_files = glob.glob(os.path.join(ds_fight_input_path, '*'))

for video_file in fight_video_files:
        video_name = os.path.basename(video_file)
        print(f'Processing {video_name}........')
        
        video_name_without_ext = os.path.splitext(video_name)[0] 
        output_video_name = f"{video_name_without_ext}.mp4"
        
        print('\t Add Noise ...')
        noise_factor = random.randint(3, 8)          
        output_video_path = os.path.join(fight_results_dir, f'noisy_{output_video_name}')        
        transform_video(video_file, output_video_path, 
                      nois_factor = noise_factor, bright_factor=None, blur=False)  
        
        print('\t Adjust Brightness ...')
        bright_factor = random.uniform(0.2, 0.7)   
        output_video_path = os.path.join(fight_results_dir, f'bright_{output_video_name}')        
        transform_video(video_file, output_video_path,
                      nois_factor=None, bright_factor=bright_factor, blur=False) 
        
        print('\t Add Noise and Adjust Brightness ...')
        noise_factor = random.randint(3, 8) 
        bright_factor = random.uniform(0.2, 0.7)   
        output_video_path = os.path.join(fight_results_dir, f'noisy_bright_{output_video_name}')        
        transform_video(video_file, output_video_path,
                      nois_factor= noise_factor, bright_factor= bright_factor, blur=False) 
        
        print('\t Apply Blur ...')         
        output_video_path = os.path.join(fight_results_dir, f'blur_{output_video_name}')        
        transform_video(video_file, output_video_path, nois_factor=None, bright_factor=bright_factor, blur=False) 
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Apply Augmentations on Normal Samples -------------------------------------------------------------------------
normal_video_files = glob.glob(os.path.join(ds_normal_input_path, '*'))

for video_file in normal_video_files:
        video_name = os.path.basename(video_file)
        print(f'Processing {video_name}........')
        
        video_name_without_ext = os.path.splitext(video_name)[0] 
        output_video_name = f"{video_name_without_ext}.mp4"
        
        print('\t Add Noise ...')
        noise_factor = random.randint(3, 8)          
        output_video_path = os.path.join(normal_results_dir, f'noisy_{output_video_name}')        
        transform_video(video_file, output_video_path, 
                      nois_factor = noise_factor, bright_factor=None, blur=False)  
        
        print('\t Adjust Brightness ...')
        bright_factor = random.uniform(0.2, 0.7)   
        output_video_path = os.path.join(normal_results_dir, f'bright_{output_video_name}')        
        transform_video(video_file, output_video_path,
                      nois_factor=None, bright_factor=bright_factor, blur=False) 
        
        print('\t Add Noise and Adjust Brightness ...')
        noise_factor = random.randint(3, 8) 
        bright_factor = random.uniform(0.2, 0.7)   
        output_video_path = os.path.join(normal_results_dir, f'noisy_bright_{output_video_name}')        
        transform_video(video_file, output_video_path,
                      nois_factor= noise_factor, bright_factor= bright_factor, blur=False) 
        
        print('\t Apply Blur ...')         
        output_video_path = os.path.join(normal_results_dir, f'blur_{output_video_name}')        
        transform_video(video_file, output_video_path, nois_factor=None, bright_factor=bright_factor, blur=False) 
#-----------------------------------------------------------------------------------------------------------------